In [1]:
import dlib
from skimage import io
from torch import FloatTensor
import torchvision.utils as vutils

In [2]:
predictor_model = "shape_predictor_68_face_landmarks.dat"
face_detector = dlib.get_frontal_face_detector()
face_pose_predictor = dlib.shape_predictor(predictor_model)

In [3]:
import torch
import torch.nn as nn
import torch.nn.parallel

def image_to_land(image):
    detected_faces = face_detector(image, 1)

    if len(detected_faces) is not 1:
        print('no face detected')

    eye_index = 20
    for i, face_rect in enumerate(detected_faces):
        pose_landmarks = face_pose_predictor(image, face_rect)

        tensor = FloatTensor(224, 224).zero_()
        points = [(it.y, it.x) for it in [pose_landmarks.part(part) for part in range(17, 68)]]
        i_min = min(i for i, _ in points)
        j_min = min(j for _, j in points)

        points = [(i - i_min, j - j_min) for i, j in points]

        i_max = max(i for i, _ in points)
        j_max = max(j for _, j in points)

        eye_i = points[eye_index][0]
        bound = int(i_max - eye_i) * 1.5 #eye_i를 기준으로 최대 bound
        ratio = 112 / bound #최대 바운드가 112 좌표로 가도록 조정해야 한다. 
        center_j = j_max // 2

        points = [(112 + (i - eye_i) * ratio, 112 + (j - center_j) * ratio) for i, j in points]

        for i, j in points:
            tensor[round(i)][round(j)] = 1
            
    return tensor.view(1, 224, 224)

In [4]:
nz     = 1000 #- feature_size     # dimension of noise vector + feature vector
nc     = 3        # number of channel - RGB
ngf    = 64       # generator 레이어들의 필터 개수를 조정하기 위한 값
ndf    = 64       # discriminator 레이어들의 필터 개수를 조정하기 위한 값
niter  = 200      # total number of epoch
lr     = 0.0002   # learning rate
beta1  = 0.5      # hyper parameter of Adam optimizer
ngpu   = 1        # number of using GPU

imageSize = 64    

class _netG(nn.Module):
    def __init__(self, ngpu):
        super(_netG, self).__init__()
        self.ngpu = ngpu
        
        self.main1 = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
        )
        self.main2 = nn.Sequential(
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )
        
        self.feature_conv = nn.Sequential(
            nn.Conv2d(1, ngf, 4, 2, 1, bias=False),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.Conv2d(ngf, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.Conv2d(ngf * 2, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.Conv2d(ngf * 4, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
        )

    def forward(self, noise, feature):
        x = self.main1(noise)
        y = self.feature_conv(feature)
        x = torch.cat([x, y], 1)
        output = self.main2(x)
        return output

In [18]:
state_dict_dir = "G_state_dict.pth"
netG = _netG(ngpu)
# netG.apply(weights_init)
netG.load_state_dict(torch.load(state_dict_dir))
netG = netG.cpu()
torch.save(netG.state_dict(), "netG_CPU.pth")

In [14]:
filename = 'test_image.jpg'
image = io.imread(filename)
landmark = image_to_land(image)

import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(imageSize),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
landmark = transform(landmark).view(1, 1, imageSize, imageSize)

from torch.autograd import Variable
noise = Variable(torch.FloatTensor(1, nz, 1, 1).normal_(0, 1))
landmark = Variable(landmark)
fake = netG(noise, landmark)

vutils.save_image(fake.data, 'test_image_land.png', normalize=True)